In [ ]:
import numpy as np
import pandas as pd
import csv
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, RepeatVector, Dense
from tensorflow.keras.layers import Reshape
from sklearn.preprocessing import MinMaxScaler

##Peaks Normalization and Log Transformation

In [ ]:
data = []
with open('CrestFactor.csv') as f:
    reader = csv.reader(f)
    next(reader) # skip header
    for row in reader:
        float_row = []
        for x in row[1:]:
            if x == '':
                float_row.append(np.nan)
            else:
                float_row.append(float(x))
        data.append(float_row)

# Replace NaN with 0
data = [np.nan_to_num(x) for x in data]

data=pd.DataFrame(data)

data = data.iloc[61:76]
timesteps = 15
n_features = data.shape[1]

In [ ]:
data

,0,1,2,3,4,5,6,7,8,9,10
61,6.57508,7.377522,2.717072,3.062202,4.023997,2.242846,3.010889,3.012142,2.893601,3.346098,2.866142
62,3.19224,3.789707,2.777794,3.074679,2.508853,1.979002,3.418515,2.574662,2.313749,2.981846,3.490789
63,2.92916,3.982395,3.068080,2.464423,2.606189,2.111822,3.031646,2.739963,2.413527,2.567403,2.649769
64,2.82780,3.204035,2.668005,2.479183,3.255711,2.048385,3.053981,2.739155,2.711618,2.537241,2.965710
65,3.12131,6.864729,3.272966,2.239624,3.080245,2.052422,3.722365,2.561038,2.645992,3.160240,3.370283
66,3.18563,6.261225,3.573301,2.367180,3.011069,2.095461,3.189913,2.868341,2.400998,2.788278,4.297873
67,2.56905,3.249932,3.539607,2.638664,3.205534,2.349518,3.970602,2.810594,2.710108,3.263634,3.512820
68,2.60964,3.257074,2.830134,2.679546,3.147938,2.289295,3.319492,2.247507,2.524131,2.510312,2.722211
69,6.54095,6.879954,2.850860,2.332262,3.050044,2.250889,2.874697,3.024104,3.099000,3.315599,3.068487
70,5.54999,6.850679,4.884897,2.352649,3.180093,2.016102,4.695638,2.689563,2.625463,4.333593,3.661671


In [ ]:
inputs = Input(shape=(timesteps, 1))
x = LSTM(15)(inputs)
# x = Dense(10)(x)
x = Dense(5)(x)
encoded = Dense(1)(x)

# decoded = Dense(16)(encoded)
# decoded = Dense(32)(decoded)
# decoded = Dense(61)(decoded)
# decoded = Reshape((61, 1))(decoded)
# decoded = LSTM(1, return_sequences=True)(decoded)
decoded = Dense(timesteps)(encoded)
decoded = Reshape((timesteps, 1))(decoded)
decoded = LSTM(15, return_sequences=True)(decoded)
# decoded = Dense(10)(decoded)
decoded = Dense(5)(decoded)
decoded = Dense(1)(decoded)

latent_space_model = Model(inputs, encoded)
autoencoder = Model(inputs, decoded)
autoencoder.compile(optimizer='adam', loss='mae')


In [ ]:
autoencoder.summary()

Model: "model_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 15, 1)]           0         
                                                                 
 lstm_10 (LSTM)              (None, 15)                1020      
                                                                 
 dense_29 (Dense)            (None, 5)                 80        
                                                                 
 dense_30 (Dense)            (None, 1)                 6         
                                                                 
 dense_31 (Dense)            (None, 15)                30        
                                                                 
 reshape_5 (Reshape)         (None, 15, 1)             0         
                                                                 
 lstm_11 (LSTM)              (None, 15, 15)            102

In [ ]:
for i in range(n_features):
    print(f'Training on column: {i+1}/{n_features}')
    x_train = data.iloc[:, i].values.reshape(-1, timesteps, 1)

    autoencoder.fit(x_train, x_train, epochs=30, batch_size=12,verbose=0)

# Save model weights
autoencoder.save_weights('autoencoder_weights.h5')

Training on column: 1/11
Training on column: 2/11
Training on column: 3/11
Training on column: 4/11
Training on column: 5/11
Training on column: 6/11
Training on column: 7/11
Training on column: 8/11
Training on column: 9/11
Training on column: 10/11
Training on column: 11/11


In [ ]:
def predict_sequence_and_latent_space(input_sequence):
    input_sequence = np.array(input_sequence)
    input_sequence = input_sequence.reshape(1, -1, 1)
    print(input_sequence.shape)
    predicted_sequence = autoencoder.predict(input_sequence)
    latent_space_output = latent_space_model.predict(input_sequence)
    return predicted_sequence.reshape(-1), latent_space_output[0][0]

column_to_predict = 1
input_sequence = data.iloc[:, column_to_predict].values.tolist()
predictions, latent_space_output = predict_sequence_and_latent_space(input_sequence)

print("Original Sequence:")
print(input_sequence)

print("Predicted Sequence:")
print(predictions)

print("Latent Space Output:")
print(latent_space_output)

(1, 15, 1)
1/1 [==============================] - 1s 662ms/step
Original Sequence:
[7.377522, 3.789707, 3.982395, 3.204035, 6.864729, 6.261225, 3.249932, 3.257074, 6.879954, 6.850679, 4.96686, 6.211209, 4.283411, 3.236666, 4.799555]
Predicted Sequence:
[2.041751  2.548183  2.8906987 2.9773386 3.0393584 3.0514944 3.157385
 2.943739  3.0816019 3.2133555 2.994978  3.090925  2.9646835 3.007577
 3.074634 ]
Latent Space Output:
7.545723


In [ ]:
import pandas as pd

# Convert the predicted sequence back to a DataFrame
predictions_df = pd.DataFrame(predictions, columns=['Predicted Sequence'])

# Get the original DataFrame
original_df = pd.DataFrame(input_sequence, columns = ['Original Sequence'])

# Get the column to predict (column_to_predict) and the corresponding column name
# column_name = original_df.columns[column_to_predict]

# Append the predicted sequence to the original DataFrame
original_df['Predicted Sequence'] = predictions_df

# Save the DataFrame to a CSV file
original_df.to_csv('predicted_sequences.csv', index=False)

print("Original Sequence and Predicted Sequence saved to 'predicted_sequences.csv'")


Original Sequence and Predicted Sequence saved to 'predicted_sequences.csv'


In [ ]:
num_columns = 11  # Total number of columns in your data

def predict_and_get_latent_output(input_sequence, column_idx):
    input_sequence = np.array(input_sequence)
    input_sequence = input_sequence.reshape(1, -1, 1)
    predicted_sequence = autoencoder.predict(input_sequence)
    latent_space_output = latent_space_model.predict(input_sequence)
    return predicted_sequence.reshape(-1), latent_space_output.reshape(-1)

# Create an array to store the latent_space_op values for each column
latent_space_op_array = []

# Loop over each column
for column_idx in range(num_columns):
    input_sequence = data.iloc[:, column_idx].values.tolist()
    predictions, latent_space_output = predict_and_get_latent_output(input_sequence, column_idx)
    latent_space_op_array.append(latent_space_output)

print(latent_space_op_array)

1/1 [==============================] - 0s 23ms/step
[array([7.5649843], dtype=float32), array([7.545723], dtype=float32), array([7.6040816], dtype=float32), array([7.648289], dtype=float32), array([7.6203547], dtype=float32), array([7.6592293], dtype=float32), array([7.6281366], dtype=float32), array([7.641325], dtype=float32), array([7.6363106], dtype=float32), array([7.588052], dtype=float32), array([7.63416], dtype=float32)]


In [ ]:
latent_space_op_array

[array([7.5649843], dtype=float32),
 array([7.545723], dtype=float32),
 array([7.6040816], dtype=float32),
 array([7.648289], dtype=float32),
 array([7.6203547], dtype=float32),
 array([7.6592293], dtype=float32),
 array([7.6281366], dtype=float32),
 array([7.641325], dtype=float32),
 array([7.6363106], dtype=float32),
 array([7.588052], dtype=float32),
 array([7.63416], dtype=float32)]

Data saved to output_data.csv


In [ ]:
data = [['Latent Space Output for each column:']]
for item in latent_space_op_array:
    data.append([item[0]])

# Write data to CSV file
csv_filename = 'latent_space_output.csv'
with open(csv_filename, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerows(data)
